In [ ]:
%pylab inline

In [ ]:
import copy
from seq_draw import rf_pulses, atoms, gradients, misc, overlays, SeqDiagram, seq_blocks

ax = plt.subplot(111)
ax.axes.set_frame_on(False)

# init sequence diagram
sd = SeqDiagram(ax=ax)

# inti with useful start values
sd.init_axes()
print sd.sqaxes.keys()

# internally axes are dictionarys which store elemnts in a list (key=atoms)
print sd.sqaxes['rf']

# define draw style
std_kw = {'linestyle': 'solid', 'color': 'black', 'linewidth': .5}
std_bold = copy.copy(std_kw)
std_bold['linewidth'] = 1.0
std_dashed = copy.copy(std_kw)
std_dashed['linestyle'] = 'dashed'
rf_kw = copy.copy(std_kw)
grad_kw = copy.copy(std_kw)
grad_kw_dashed = copy.copy(grad_kw)
grad_kw_dashed['linestyle'] = 'dashed'
epi_kw = copy.copy(grad_kw)
# epi_kw['alpha'] = 0.5
epi_caipi_kw = copy.copy(grad_kw)
epi_caipi_kw['color'] = 'red'
epi_caipi_kw['linestyle'] = 'dotted'
vline_kw = copy.copy(std_kw)
vline_kw['linewidth'] = .5
vline_kw['linestyle'] = 'dashed'
span_yoff = 0.925
arrow_kw={'head_width': 0.02, 'head_length': 0.05, 'color': 'black'}

# define times
start_delay = 0.75
echo_time = 5.0
nref = 1
rf_pulse_dur = 0.25
rf_intensity = 0.1
rf_int_wet = 0.15
grad_read_dur = 0.1
grad_read_int = 0.025
grad_blip_dur = 0.05
crush_dur = 0.1
crush_int = 3*grad_read_int
enc_dur = grad_read_dur
enc_int = 0.05
nav_rep = 3
epi_rep = 9

## --- Create pulse objects ---

# use fa scalefactors from WET publication
wet_scaling = [(81.4**0.5)/(161.0**0.5), (101.4**0.5)/(161.0**0.5), (69.0**0.5)/(161.0**0.5), (161.0**0.5)/(161.0**0.5)]
wet1 = rf_pulses.SincPulse(sd, 'rf', rf_pulse_dur, rf_int_wet*wet_scaling[0], side_lobes=5, plot_kw=rf_kw)
wet2 = rf_pulses.SincPulse(sd, 'rf', rf_pulse_dur, rf_int_wet*wet_scaling[1], side_lobes=5, plot_kw=rf_kw)
wet3 = rf_pulses.SincPulse(sd, 'rf', rf_pulse_dur, rf_int_wet*wet_scaling[2], side_lobes=5, plot_kw=rf_kw)
wet4 = rf_pulses.SincPulse(sd, 'rf', rf_pulse_dur, rf_int_wet*wet_scaling[3], side_lobes=5, plot_kw=rf_kw)
wet_slisel = gradients.RectGrad(sd, 'gz', rf_pulse_dur, 0.5*grad_read_int, plot_kw=grad_kw)
wet_spoil = gradients.RectGrad(sd, 'gz', rf_pulse_dur/2, -2*grad_read_int, plot_kw=grad_kw)
wet_delta_spoil = misc.Line(sd, 'gz', rf_pulse_dur/2, plot_kw=grad_kw)
wet_rf = [wet1, wet_delta_spoil, wet2, wet_delta_spoil, wet3, wet_delta_spoil, wet4, wet_delta_spoil]
wet_rf = [wet1, wet_delta_spoil]
wet_gz = [wet_slisel, wet_spoil] #* 4

# PCASL
pcasl_rep = 4
pcasl_int = 0.05
pcasl_rf_dur = 0.125
pcasl_ss_dur = 2 * pcasl_rf_dur
pcasl_ss = gradients.TrapGrad(sd, 'gz', pcasl_ss_dur, 0.5*grad_read_int, plot_kw=grad_kw)
pcasl_ssi = gradients.TrapGrad(sd, 'gz', pcasl_ss_dur/2, -grad_read_int, plot_kw=grad_kw)
pcasl_rf_pulse = rf_pulses.HanningPulse(sd, 'rf', pcasl_rf_dur, pcasl_int, plot_kw=rf_kw)
pcasl_rf_space = misc.Line(sd, 'rf', pcasl_rf_dur/2, plot_kw=std_kw)
pcasl_rf_element = [pcasl_rf_space, pcasl_rf_pulse, pcasl_rf_space, pcasl_rf_space, pcasl_rf_space]
pcasl_gz_element = [pcasl_ss, pcasl_ssi]
pcasl_delay = misc.Line(sd, 'gz', sd.get_total_duration(pcasl_rf_element), plot_kw=std_dashed)
pcasl_rf =  pcasl_rf_element * int(pcasl_rep/2) + [pcasl_delay] + pcasl_rf_element * (pcasl_rep - (int(pcasl_rep/2) + 1))
pcasl_gz =  pcasl_gz_element * int(pcasl_rep/2) + [pcasl_delay] + pcasl_gz_element * (pcasl_rep - (int(pcasl_rep/2) + 1))

# Background suppression
bs_pulse_dur = 0.25
bs_int = 0.125
bs_rf_pulse = rf_pulses.HyperbolicSecantPulse(sd, 'rf', bs_pulse_dur, bs_int, plot_kw=rf_kw)
bs_ss_dur = 2 * bs_pulse_dur
bs_spoil_dur = bs_pulse_dur
bs_spoil_int = 0.075
bs_ss = gradients.TrapGrad(sd, 'gz', bs_ss_dur, grad_read_int*0.5, plot_kw=grad_kw)
bs_spoil = gradients.TrapGrad(sd, 'gz', bs_spoil_dur, bs_spoil_int, plot_kw=grad_kw)
bs_del = misc.Line(sd, 'rf', bs_pulse_dur/2, plot_kw=std_kw)
bs_intrr = misc.Interruption(sd, 'rf', bs_pulse_dur, 0.025, plot_kw=std_kw)
bs_rf = [bs_intrr, bs_del, bs_rf_pulse, [bs_del] * 3, bs_intrr]
bs_gxy = [bs_intrr, [bs_del] * 6, bs_intrr]
bs_gz = [bs_intrr, bs_ss, bs_spoil, bs_intrr]

## GRASE
# rf pulses
exc = rf_pulses.SincPulse(sd, 'rf', rf_pulse_dur, rf_intensity, label=u'90˚', side_lobes=6, plot_kw=rf_kw)
ref = rf_pulses.SincPulse(sd, 'rf', rf_pulse_dur, 1.5*rf_intensity, label=u'180˚', label_coord=(-0.125, 0.15), plot_kw=rf_kw)

# gradients
slab_sel = gradients.RectGrad(sd, 'gz', rf_pulse_dur, grad_read_int, plot_kw=grad_kw)
slab_sel_reph = gradients.RectGrad(sd, 'gz', rf_pulse_dur/2, -grad_read_int*2, plot_kw=grad_kw)
crush_grad = gradients.RectGrad(sd, 'gz', crush_dur, crush_int, plot_kw=grad_kw)
read_enc = gradients.TrapGrad(sd, 'gx', enc_dur, grad_read_int * 0.5, plot_kw=grad_kw)
pe1_enc = gradients.TrapGrad(sd, 'gy', enc_dur, -grad_read_int*1.5, plot_kw=grad_kw)
pe2_enc = gradients.TrapGradTable(sd, 'gz', enc_dur, enc_int, -enc_int, tab_type=gradients.GradTableType.CENTER_OUT,
                                  ft_ramp_ratio=1., plot_kw=grad_kw, draw_arrow=False)
# navigators
pc_scans = seq_blocks.get_epi_readout(sd, 'gy', nav_rep, grad_read_dur, grad_read_int, read_grad=slab_sel, plot_kw=grad_kw)

# EPI readout
epi_read = seq_blocks.get_epi_readout(sd, 'gx', epi_rep, grad_read_dur, grad_read_int, read_grad=pe1_enc, plot_kw=grad_kw)
blip = gradients.BlipGrad(sd, 'gy', grad_blip_dur, grad_read_int, plot_kw=grad_kw)
blip_grappa = gradients.BlipGrad(sd, 'gy', grad_blip_dur, grad_read_int, plot_kw=epi_kw)
blip_caipi = gradients.BlipGrad(sd, 'gy', grad_blip_dur, grad_read_int, plot_kw=epi_caipi_kw)
epi_pe1 = seq_blocks.get_epi_blips(sd, 'gy', epi_rep - 1, grad_blip_dur, grad_read_int*1.5, grad_read_dur, blip_grad=blip, blip_pattern=[1.0], plot_kw=grad_kw)
epi_pe2 = seq_blocks.get_epi_blips(sd, 'gz', epi_rep - 1, grad_blip_dur, grad_read_int*0, grad_read_dur, blip_pattern=[1.], blip_grad=blip_grappa, plot_kw=epi_kw)
epi_pe2_CAIPI = seq_blocks.get_epi_blips(sd, 'gz', epi_rep - 1, grad_blip_dur, grad_read_int, grad_read_dur, blip_pattern=[1., -1.], blip_grad=blip_caipi, plot_kw=epi_caipi_kw)

# define some time spans
delta = misc.Line(sd, 'rf', 0.5, plot_kw=std_kw)
delta_rf = misc.Line(sd, 'gy', rf_pulse_dur, plot_kw=std_kw)
delta_crush = misc.Line(sd, 'rf', crush_dur, plot_kw=std_kw)

# define some useful elements
refocusing_rf = [delta_crush, ref, delta_crush]
refocusing_gxy = [crush_grad, delta_rf, crush_grad]
refocusing_gz = [crush_grad, slab_sel, crush_grad]

# --- Timing ---

wet_duration = sd.get_total_duration(wet_rf)
pcasl_dur = sd.get_total_duration(pcasl_rf)
bs_dur = sd.get_total_duration(bs_rf)
epi_duration = sd.get_total_duration(epi_read)
refocusing_duration = sd.get_total_duration(refocusing_rf)
echo_time = 2. * (epi_duration/2 + refocusing_duration/2 + enc_dur)
wet_start_time = start_delay
pcasl_start_time = wet_start_time + wet_duration
bs_start_time = pcasl_start_time + pcasl_dur
exc_center_time = bs_start_time + bs_dur + rf_pulse_dur/2

## --- Draw sequence ---

## Add pulses to axes
sd.fill(tp=start_delay, plot_kw=std_kw)
# presaturation
sd.add_atom(wet_rf, 'rf')
sd.add_atom(wet_gz, 'gz')
sd.fill(plot_kw=std_kw)
# pcasl
sd.add_atom([pcasl_rf], 'rf')
sd.add_atom([pcasl_gz], 'gz')
sd.fill(plot_kw=std_kw)
# bs
sd.add_atom([bs_rf], 'rf')
sd.add_atom([bs_gxy], 'gx')
sd.add_atom([bs_gxy], 'gy')
sd.add_atom([bs_gz], 'gz')
sd.fill(plot_kw=std_kw)
# Start with excitation and navigators
sd.add_atom([exc], 'rf')
sd.add_atom([delta_rf], 'gx')
sd.add_atom([delta_rf, misc.Line(sd, 'gy', rf_pulse_dur/2, plot_kw=std_kw), pc_scans], 'gy')
sd.add_atom([slab_sel, slab_sel_reph], 'gz')
sd.fill(axes=['rf','gx', 'gy', 'gz'], tp=start_delay + wet_duration + pcasl_dur + bs_dur + rf_pulse_dur/2 + echo_time/2 - crush_dur - rf_pulse_dur/2, plot_kw=std_kw)
# refocusing and EPI trajectory
sd.add_atom(refocusing_rf, 'rf')
sd.add_atom([refocusing_gxy, read_enc, epi_read,read_enc], 'gx')
sd.add_atom([refocusing_gxy, pe1_enc, epi_pe1,pe1_enc], 'gy')
sd.add_atom([refocusing_gz, pe2_enc, epi_pe2,pe2_enc], 'gz')
sd.fill(plot_kw=std_kw)
# 2nd refocusing
sd.add_atom(refocusing_rf, 'rf')
sd.add_atom(refocusing_gxy, 'gx')
sd.add_atom(refocusing_gxy, 'gy')
sd.add_atom(refocusing_gz, 'gz')
sd.fill(plot_kw=std_kw)
sd.fill(tp=sd.get_total_duration(sd.sqaxes['rf']['atoms']) + 0.25, plot_kw=std_dashed)

## --- Add overlay elements ---

# add echo time arrow
te_span = overlays.TimeSpan(sd, exc_center_time, echo_time, span_yoff, 'TE', arrow_kw=arrow_kw)
te_span.draw()
vline1 = overlays.VerticalLine(sd, exc_center_time, plot_kw=vline_kw, ymax=span_yoff)
vline1.draw()
vline2 = overlays.VerticalLine(sd, exc_center_time + echo_time, plot_kw=vline_kw, ymax=span_yoff)
vline2.draw()
vline3 = overlays.VerticalLine(sd, pcasl_start_time, plot_kw=vline_kw, ymax=span_yoff)
vline3.draw()
vline4 = overlays.VerticalLine(sd, bs_start_time, plot_kw=vline_kw, ymax=span_yoff)
vline4.draw()
pld_span = overlays.TimeSpan(sd, bs_start_time, bs_dur + rf_pulse_dur/2, span_yoff, 'PLD', arrow_kw=arrow_kw)
pld_span.draw()
pld_span = overlays.TimeSpan(sd, pcasl_start_time, pcasl_dur, span_yoff, 'LD', arrow_kw=arrow_kw)
pld_span.draw()

# Shaded areas
wet_area = overlays.VerticalSpan(sd, wet_start_time, wet_start_time + wet_duration, plot_kw={'color': 'green'})
wet_area.draw()
pcasl_area = overlays.VerticalSpan(sd, pcasl_start_time, pcasl_start_time + pcasl_dur)
pcasl_area.draw()
bs_area = overlays.VerticalSpan(sd, bs_start_time + bs_intrr.duration, bs_start_time + bs_dur, plot_kw={'color': 'green'})
bs_area.draw()

# Brackets around WET
wet_bra = overlays.Bra(sd, wet_start_time, xlen=0.1, plot_kw=std_bold)
wet_ket = overlays.Ket(sd, wet_start_time + wet_duration, xlen=0.1, plot_kw=std_bold)
wet_bra.draw()
wet_ket.draw()

# Brackets around BS
bs_bra = overlays.Bra(sd, bs_start_time + bs_intrr.duration, xlen=0.1, plot_kw=std_bold)
bs_ket = overlays.Ket(sd, bs_start_time + bs_dur, xlen=0.1, plot_kw=std_bold)
bs_bra.draw()
bs_ket.draw()

# draw
sd.draw(debug=False)

# save

# fig.savefig('3DGRASE.pdf', bbox_inches='tight')

In [ ]:
# SeqDiagram shows all axes with its elements when printed
sd